In [1]:
from pyrootmemo.materials import SingleRoot, MultipleRoots
from pyrootmemo.materials import Soil
from pyrootmemo.models import Waldron1977
import numpy as np
import matplotlib.pyplot as plt

In [3]:
barley_single = SingleRoot(species="hordeum vulgare", elastic_modulus=2E6, diameter=1E-2)
barley_multiple = MultipleRoots(species="hordeum vulgare", elastic_modulus=2E6, diameter=[0.01, 0.02, 0.03])

In [4]:
loam = Soil(type="clay_loam", unit_weight={"bulk":1.20*9.81, "saturated":20}, friction_angle=37,cohesion=2.3)